In [ ]:
!pip install -q evaluate jiwer torch faster-whisper transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 

In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset, Audio
import evaluate
from functools import partial
from faster_whisper import WhisperModel

Please be sure to select "True" whether you're using the FasterWhisper version of the model or "False" if not.

In [ ]:
using_faster_whisper = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "openai/whisper-small"

In [ ]:
if using_faster_whisper:
    !ct2-transformers-converter --model {model_id} --output_dir model_vlad --copy_files tokenizer_config.json preprocessor_config.json --quantization float16
    model = WhisperModel("model_vlad", device=str(device), compute_type="float16")

else:
    model = WhisperForConditionalGeneration.from_pretrained(model_id).to(device)
    model.generate = partial(model.generate, language="ro", task="transcribe")
    processor = WhisperProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def is_audio_short(sample, max_duration=30):
    audio = sample["audio"]
    audio_array = audio["array"]
    sampling_rate = audio["sampling_rate"]
    duration_seconds = len(audio_array) / sampling_rate
    return duration_seconds <= max_duration

dataset = load_dataset("google/fleurs", "ro_ro", split="test", trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset = dataset.filter(is_audio_short)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/883 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/882 [00:00<?, ? examples/s]

In [ ]:
def transcribe(batch):

    audio = batch["audio"]["array"]

    if using_faster_whisper:
        segments, info = model.transcribe(audio, beam_size=5, language="ro", condition_on_previous_text=False)

        transcription = ""
        for segment in segments:
            transcription += segment.text

    else:
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(device)
        predicted_ids = model.generate(inputs)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    print(transcription)
    return transcription

In [ ]:
predictions = []
references = []
norm_predictions = []
norm_references = []

print("now we start")

for sample in dataset:
    transcription = transcribe(sample)
    predictions.append(transcription)
    references.append(sample["raw_transcription"])

now we start
 Înainte de sosiera trupe lor, haiti il mai avusese vinani 800, probleme legate de boala.
 Pe tot parcursul anilor 1960, Brezinskii a lucrat pentru John F. Kennedy în calitate de concilier al acestuia, iar apoi pentru administrația Lyndon B. Johnson.
 Compoziția acestor cristale se potrivește cu acelor întâlnite în urina animalelor de companie afectate, atunci când sunt comparate prin o spectroscopie în infrarosu, spectroscopie în IR.
 Fenomenile climatici este severe regionale si dizonieri include viscole, furtun de zapata, furtun de chiatasi, furtun de praf.
 Celal de numilă analiză vincul cel mai bun film, regizor imagine, design de costume, montaj de film, coloanțonoră, design de producție, regie de suneni, mixtașo noși și scenariu original.
 Astăzi s-a format în oceanul Atlantic, furtuna subtropicală Gerii, a 10-a furtună care a privit un nume propriu din sezonul Uraganelor Atlantice.
 Cu oameni, cu un vârst de 53 de ani, a început cuvernarea a început cu acestui an ș

In [ ]:
wer_metric = evaluate.load("wer")
wer = round(100 * wer_metric.compute(references=references, predictions=predictions), 2)
print("WER : ", wer)